In [250]:
import geopandas as gpd
import pandas as pd
import folium
import re
from folium.plugins import MarkerCluster
import os
from folium.plugins import TimestampedGeoJson

In [ ]:
def load_data():
    # DETER alerts_df_df_df (GEODF)
    alerts_df = gpd.read_file('../data/deter-amz-public-2024out08/deter-amz-deter-public.shp', encoding='utf-8')
    alerts_df.loc[alerts_df['CLASSNAME'] == 'DEGRDACAO', 'CLASSNAME'] = 'DEGRADACAO'
    alerts_df = alerts_df[~(alerts_df['CLASSNAME'] == 'CORTE_SELETIVO')]
    alerts_df['VIEW_DATE'] = pd.to_datetime(alerts_df['VIEW_DATE'])
    alerts_df['ANO'] = alerts_df['VIEW_DATE'].dt.year
    alerts_df['MES'] = alerts_df['VIEW_DATE'].dt.month
    alerts_df['MES/ANO'] = alerts_df['VIEW_DATE'].dt.strftime('%Y-%m')

    # DETER ALERTS (CSV)
    df_deter = pd.DataFrame(alerts_df)
    df_deter = df_deter.drop(columns=['FID', 'QUADRANT', 'PATH_ROW', 'SENSOR', 'SATELLITE', 'geometry'])

    
    #IBGE DATA
    legal_amazon = gpd.read_file('../data/brazilian_legal_amazon/brazilian_legal_amazon.shp',encoding='utf-8')
    states = gpd.read_file('../data/states_legal_amazon/states_legal_amazon.shp',encoding='utf-8')
    
    ac = gpd.read_file('../data/malhas_regionais_ibge/AC_Municipios_2022/AC_Municipios_2022.shp', encoding='utf-8')
    am = gpd.read_file('../data/malhas_regionais_ibge/AM_Municipios_2022/AM_Municipios_2022.shp', encoding='utf-8')
    ap = gpd.read_file('../data/malhas_regionais_ibge/AP_Municipios_2022/AP_Municipios_2022.shp', encoding='utf-8')
    ma = gpd.read_file('../data/malhas_regionais_ibge/MA_Municipios_2022/MA_Municipios_2022.shp', encoding='utf-8')
    mt = gpd.read_file('../data/malhas_regionais_ibge/MT_Municipios_2022/MT_Municipios_2022.shp', encoding='utf-8')
    pa = gpd.read_file('../data/malhas_regionais_ibge/PA_Municipios_2022/PA_Municipios_2022.shp', encoding='utf-8')
    ro = gpd.read_file('../data/malhas_regionais_ibge/RO_Municipios_2022/RO_Municipios_2022.shp', encoding='utf-8')
    rr = gpd.read_file('../data/malhas_regionais_ibge/RR_Municipios_2022/RR_Municipios_2022.shp', encoding='utf-8')
    to = gpd.read_file('../data/malhas_regionais_ibge/TO_Municipios_2022/TO_Municipios_2022.shp', encoding='utf-8')

    df_states = pd.concat([ac, am, ap, ma, mt, pa, ro, rr, to])

    df_states.rename(columns={'CD_MUN':'GEOCODIBGE'}, inplace=True)

    c_units = gpd.read_file('../data/conservation_units_legal_amazon/conservation_units_legal_amazon.shp',encoding='utf-8')
    c_units.rename(columns={'nome':'UC'},inplace=True)
    
    return alerts_df, df_deter, legal_amazon, states, df_states, c_units

alerts_df, df_deter, legal_amazon, states, df_states, c_units = load_data()
folder_path = "../Visualizations/DETER/Maps"
for file in os.listdir(folder_path):
    os.remove(os.path.join(folder_path,file))



In [290]:
df_deter["UF"].unique()

array(['PA', 'MT', 'AM', 'RO', 'MA', 'TO', 'RR', 'AP', 'AC'], dtype=object)

In [295]:
dfs_deter = {uf: df_deter[df_deter['UF'] == uf] for uf in df_deter['UF'].unique()}

df_deter_PA = dfs_deter['PA']
df_deter_MT = dfs_deter['MT']
df_deter_AM = dfs_deter['AM']
df_deter_RO = dfs_deter['RO']
df_deter_MA = dfs_deter['MA']
df_deter_TO = dfs_deter['TO']
df_deter_RR = dfs_deter['RR']
df_deter_AP = dfs_deter['AP']
df_deter_AC = dfs_deter['AC']

df_deter_PA.head()

,CLASSNAME,VIEW_DATE,AREAUCKM,UC,AREAMUNKM,MUNICIPALI,GEOCODIBGE,UF,ANO,MES,MES/ANO
0,CICATRIZ_DE_QUEIMADA,2018-01-11,0.0,None,0.459839,Monte Alegre,1504802,PA,2018,1,2018-01
1,CICATRIZ_DE_QUEIMADA,2018-01-14,0.0,None,0.340975,Itaituba,1503606,PA,2018,1,2018-01
3,DESMATAMENTO_VEG,2018-01-14,0.0,None,0.070781,Altamira,1500602,PA,2018,1,2018-01
4,CICATRIZ_DE_QUEIMADA,2018-01-14,0.0,None,0.149432,Itaituba,1503606,PA,2018,1,2018-01
5,DESMATAMENTO_CR,2018-01-14,0.0,None,0.136849,Trairao,1508050,PA,2018,1,2018-01


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Preparing the data
df_deter_PA.reset_index(drop=True, inplace=True)

# Selecting relevant columns for time series forecasting
data = df_deter_PA[['AREAMUNKM']].values

# Scaling the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Creating sequences for LSTM
def create_sequences(data, seq_length=12):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12  # Using 12 months for yearly prediction
X, y = create_sequences(data_scaled, seq_length)

# Splitting the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Building the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', return_sequences=True, input_shape=(seq_length, 1)),
    tf.keras.layers.LSTM(50, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Training the model
model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)

# Predicting the next year's data
predicted = []
last_sequence = data_scaled[-seq_length:]  # Last 12 months as input for next year's prediction

for _ in range(12):
    next_value = model.predict(last_sequence[np.newaxis, :, :], verbose=0)
    predicted.append(next_value[0, 0])
    last_sequence = np.append(last_sequence[1:], next_value, axis=0)

# Inverse scaling to get actual values
predicted = scaler.inverse_transform(np.array(predicted).reshape(-1, 1)).flatten()

# Preparing the resulting DataFrame
future_dates = pd.date_range(start="2025-01", periods=12, freq='M')
future_df = pd.DataFrame({
    'CLASSNAME': ['PREDICTED'] * 12,
    'VIEW_DATE': future_dates,
    'AREAUCKM': [0.0] * 12,
    'UC': [None] * 12,
    'AREAMUNKM': predicted,
    'MUNICIPALI': ['Monte Alegre'] * 12,  # Placeholder
    'GEOCODIBGE': [1504802] * 12,  # Placeholder
    'UF': ['PA'] * 12,
    'ANO': [2025] * 12,
    'MES': list(range(1, 13)),
    'MES/ANO': future_dates.strftime('%Y-%m')
})


In [252]:
print(alerts_df)

                FID             CLASSNAME QUADRANT PATH_ROW  VIEW_DATE SENSOR  \
0       100002_hist  CICATRIZ_DE_QUEIMADA     None   170105 2018-01-11   AWFI   
1       100003_hist  CICATRIZ_DE_QUEIMADA     None   169105 2018-01-14   AWFI   
2       100005_curr  CICATRIZ_DE_QUEIMADA     None   037017 2024-09-26    WFI   
3       100005_hist      DESMATAMENTO_VEG     None   169105 2018-01-14   AWFI   
4       100006_hist  CICATRIZ_DE_QUEIMADA     None   169105 2018-01-14   AWFI   
...             ...                   ...      ...      ...        ...    ...   
392804    9999_curr       DESMATAMENTO_CR     None   038016 2023-09-29    WFI   
392805     999_curr  CICATRIZ_DE_QUEIMADA     None   036016 2023-08-16    WFI   
392806      99_hist            DEGRADACAO        D   321074 2016-08-02  AWIFS   
392807       9_curr       DESMATAMENTO_CR     None   036016 2023-08-01    WFI   
392808       9_hist  CICATRIZ_DE_QUEIMADA        B   324078 2016-08-17  AWIFS   

            SATELLITE  AREA

In [253]:
print("DF Deter", df_deter)
print("\nLegal Amazon", legal_amazon)
print("\nStates", states)

DF Deter                    CLASSNAME  VIEW_DATE  AREAUCKM    UC  AREAMUNKM  \
0       CICATRIZ_DE_QUEIMADA 2018-01-11       0.0  None   0.459839   
1       CICATRIZ_DE_QUEIMADA 2018-01-14       0.0  None   0.340975   
2       CICATRIZ_DE_QUEIMADA 2024-09-26       0.0  None   1.373554   
3           DESMATAMENTO_VEG 2018-01-14       0.0  None   0.070781   
4       CICATRIZ_DE_QUEIMADA 2018-01-14       0.0  None   0.149432   
...                      ...        ...       ...   ...        ...   
392804       DESMATAMENTO_CR 2023-09-29       0.0  None   0.206886   
392805  CICATRIZ_DE_QUEIMADA 2023-08-16       0.0  None   0.425538   
392806            DEGRADACAO 2016-08-02       0.0  None  22.043149   
392807       DESMATAMENTO_CR 2023-08-01       0.0  None   0.139690   
392808  CICATRIZ_DE_QUEIMADA 2016-08-17       0.0  None   7.582060   

                MUNICIPALI GEOCODIBGE  UF   ANO  MES  MES/ANO  
0             Monte Alegre    1504802  PA  2018    1  2018-01  
1                 Itai

In [254]:
print("DF Municipalities", df_states)
print("\nC Units", c_units)

DF Municipalities     GEOCODIBGE          NM_MUN SIGLA_UF  AREA_KM2  \
0      1200013      Acrelândia       AC  1811.613   
1      1200054    Assis Brasil       AC  4979.073   
2      1200104       Brasiléia       AC  3928.174   
3      1200138          Bujari       AC  3034.869   
4      1200179        Capixaba       AC  1705.824   
..         ...             ...      ...       ...   
134    1721208  Tocantinópolis       TO  1083.600   
135    1721257        Tupirama       TO   706.883   
136    1721307      Tupiratins       TO   889.126   
137    1722081    Wanderlândia       TO  1365.431   
138    1722107         Xambioá       TO  1190.489   

                                              geometry  
0    POLYGON ((-67.07612 -10.08798, -67.07659 -10.0...  
1    POLYGON ((-69.55253 -10.87353, -69.52086 -10.8...  
2    POLYGON ((-68.75712 -11.01097, -68.75752 -11.0...  
3    POLYGON ((-67.92167 -9.69355, -67.91736 -9.693...  
4    POLYGON ((-67.73403 -10.71177, -67.73414 -10.7...  
.. 

In [255]:
# LOADING TEXTS (ENGLISH AND PORTUGUESE)

df_texts = pd.read_csv('../texts/texts_deter.csv', sep='§', engine='python')
english = {list(df_texts['Key'])[i]: list(df_texts['English'])[i] for i in range(len(list(df_texts['Key'])))}
# portuguese = {list(df_texts['Key'])[i]: list(df_texts['Portuguese'])[i] for i in range(len(list(df_texts['Key'])))}

classes_deter_en = {'CICATRIZ_DE_QUEIMADA': 'Forest Fire Scar',
          'DESMATAMENTO_CR': 'Deforestation with Exposed Soil',
          'DESMATAMENTO_VEG': 'Deforestation with Vegetation',
          'MINERACAO': 'Mining',
          'DEGRADACAO': 'Degradation',
          'CS_DESORDENADO': 'Selective Logging Type 1 (Disordered)',
          'CS_GEOMETRICO': 'Selective Logging Type 2 (Geometric)',
}

states_dict = {
    "MT": "Mato Grosso",
    "PA": "Pará",
    "AM": "Amazonas",
    "RO": "Rondônia",
    "MA": "Maranhão",
    "RR": "Roraima",
    "AC": "Acre",
    "TO": "Tocantins",
    "AP": "Amapá"
}

def get_texts():
    return classes_deter_en, english



dict_classes, texts = get_texts()

In [256]:
print(dict_classes)

{'CICATRIZ_DE_QUEIMADA': 'Forest Fire Scar', 'DESMATAMENTO_CR': 'Deforestation with Exposed Soil', 'DESMATAMENTO_VEG': 'Deforestation with Vegetation', 'MINERACAO': 'Mining', 'DEGRADACAO': 'Degradation', 'CS_DESORDENADO': 'Selective Logging Type 1 (Disordered)', 'CS_GEOMETRICO': 'Selective Logging Type 2 (Geometric)'}


In [257]:
print(texts)

{'page_title': 'Amazon Deforestation Monitor', 'date_format': '%m-%d-%Y', 'date_format2': '%m-%Y', 'date_format3': 'MM/DD/YYYY', 'deter_expander_title': 'What is DETER?', 'deter_expander_desc_1': 'The [DETER](http://www.inpe.br/cra/projetos_pesquisas/deter.php) system, short for "Real-Time Deforestation Detection", is a system developed by the Brazilian National Institute for Space Research (INPE), aimed at monitoring and identifying changes in forest cover within the Legal Amazon in Brazil. This system is essential for environmental enforcement and combating illegal deforestation, providing crucial data for prevention and control actions.\nThrough high-resolution satellite images, DETER can detect areas of deforestation and forest degradation with high precision and almost in real-time. This information is made available periodically, allowing for a swift response from environmental authorities.\n\nDETER\'s detections are classified into various categories, reflecting different types 

In [258]:
def get_centroids(geo_df, mode=2, crs='EPSG:31982'):

    if mode==0:
        centroids = geo_df.copy()
        centroids["centroid"] = centroids.geometry.centroid
        centroids["latitude"] = centroids.centroid.y
        centroids["longitude"] = centroids.centroid.x
        return centroids

    if mode==1:
        geo_df_proj = geo_df.copy()
        geo_df_proj.to_crs(crs)
        
        geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
        geo_df_proj['latitude'] = geo_df_proj.centroid.y
        geo_df_proj['longitude'] = geo_df_proj.centroid.x
        
        centroids = gpd.GeoDataFrame(geo_df_proj, geometry='centroid', crs=crs)
    
        centroids = centroids.to_crs(geo_df.crs)
        return centroids

    if mode==2:
        df = geo_df.copy()
        df['representative_point'] = df.geometry.representative_point()
        df['latitude'] = df['representative_point'].apply(lambda p: p.y)
        df['longitude'] = df['representative_point'].apply(lambda p: p.x)
        return df

def folium_map_init():
    map = folium.Map(location=[-7.25, -60], zoom_start=4)
    return map

def folium_add_markers(container, df_data, geo_df, get_centroid_mode, df_deter, key, popup_title_column, popup_total_area_text='Área Total Afetada:',total_area_column='AREAMUNKM'):

    all_classes = sorted(df_deter['CLASSNAME'].unique())
    df_centroids = get_centroids(geo_df,get_centroid_mode)
    
    for idx, row in df_data.iterrows():

        coords = df_centroids.loc[df_centroids[key] == row[key]].iloc[0]

        info = ''
        
        if row['AREAMUNKM']>0:
            df_stats = df_deter[df_deter[key] == row[key]]
            df_stats_summed = df_stats.groupby('CLASSNAME')['AREAMUNKM'].sum().reset_index()
            df_stats_complete = pd.DataFrame({'CLASSNAME': all_classes})
            df_stats_complete = df_stats_complete.merge(df_stats_summed, on='CLASSNAME', how='left').fillna(0)
            df_stats_complete['DESC'] = df_stats_complete['CLASSNAME'].map(dict_classes)
            df_stats_complete = df_stats_complete.sort_values(by='AREAMUNKM', ascending=False)
    
            total = df_stats_complete['AREAMUNKM'].sum()
            
            # Calculates percentage of every class
            for ind, lin in df_stats_complete.iterrows():
                perc = (lin['AREAMUNKM'] * 100) / total
                info += f"{lin['DESC']}: {lin['AREAMUNKM']:.0f} km² ({perc:.2f}%)<br>"
        
        popup_text = f"""
        <div style='white-space: nowrap;'>
        <span style='font-size: 16px; font-weight: bold;'>{row[popup_title_column]}</span><br><br> {popup_total_area_text} {row[total_area_column]:.0f} km²<br><br> {info}
        </div>
        """

        # Add marker on Map or MarkerCluster (container)
        folium.Marker(
            location=[coords['latitude'], coords['longitude']],
            popup=popup_text,
            icon=folium.Icon(color='red', icon='triangle-exclamation', prefix='fa')
        ).add_to(container)
    
    return container

In [ ]:
def save_map(file_name,map):
    map.save(f"../Visualizations/DETER/Maps/{file_name}.html")

### States Map:

In [260]:
def states_map():
    ############# Data Preparation #############
    # Load shapefiles for regions (if not already loaded)
    # legal_amazon = gpd.read_file('data/brazilian_legal_amazon/brazilian_legal_amazon.shp', encoding='utf-8')
    # states = gpd.read_file('data/states_legal_amazon/states_legal_amazon.shp', encoding='utf-8')
    
    df_deter = alerts_df.copy()
    gb_uf = df_deter.groupby('UF')['AREAMUNKM'].sum().sort_values(ascending=False)
    gb_uf = pd.DataFrame(gb_uf)
    gb_uf['NOME_ESTADO'] = gb_uf.index.map(states_dict)
    gb_uf['NOME_SIGLA'] = gb_uf['NOME_ESTADO'] + ' (' + gb_uf.index + ')' 
    gb_uf = gb_uf.reset_index()

    states_copy = states.copy()
    states_copy = states_copy.rename(columns={'sigla': 'UF'})

    #############       Folium       #############
    map = folium_map_init()

    # Customizing state borders color
    style_states = {'fillOpacity': 0.3, 'color': '#005f73', 'weight': 2}
    folium.GeoJson(states_copy, name='States', style_function=lambda x: style_states).add_to(map)

    # Customizing Legal Amazon border color
    style_legal_amazon = {'fillOpacity': 0, 'color': '#0a9396', 'weight': 3}
    folium.GeoJson(legal_amazon, name='Legal Amazon', style_function=lambda x: style_legal_amazon).add_to(map)

    # Customizing Choropleth color scheme
    folium.Choropleth(
        geo_data=states_copy,
        data=gb_uf,
        columns=['UF', 'AREAMUNKM'],
        key_on='feature.properties.UF',
        fill_color='Reds',
        fill_opacity=0.7,
        line_opacity=0.2,
        nan_fill_color='white',
        bins=8,
        highlight=True,
        legend_name='Affected Area in km²',
        name='Most Affected States'
    ).add_to(map)

    # Adding markers with popups for more information
    map = folium_add_markers(map, gb_uf, states_copy, 1, df_deter, 'UF', 'NOME_SIGLA', texts['total_dmg'], 'AREAMUNKM')

    # Adding layer control
    folium.LayerControl().add_to(map)

    return map


In [261]:
dict_classes, texts = get_texts()
map = states_map()
save_map('States_EN',map)

/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/1685801861.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/1685801861.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['latitude'] = geo_df_proj.centroid.y
/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/1685801861.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['longitude'] = geo_df_proj.centroid.x


### Cities Map

In [262]:
def cities_map(filter=[]):
    
    ############# Data Preparation #############

    geocodibge = alerts_df.drop_duplicates(subset='MUNICIPALI').set_index('MUNICIPALI')['GEOCODIBGE']
    sum_areamunkm = alerts_df.groupby('MUNICIPALI')['AREAMUNKM'].sum().reset_index()
    sum_areamunkm['GEOCODIBGE'] = sum_areamunkm['MUNICIPALI'].map(geocodibge)

    merge = pd.merge(df_states, sum_areamunkm, on='GEOCODIBGE', how='left')

    if len(filter)>0:
        merge = merge[merge['SIGLA_UF'].isin(filter)]
    
    # merge[merge['AREAMUNKM'].isna()]
    # 226 cidades não contém avisos. Esses valores ausentes serão preenchidos com 0.
    merge['AREAMUNKM'].fillna(0, inplace=True)

    #############       Folium       #############
    map = folium_map_init()

    style_cities = {'fillOpacity':0 ,'color' : '#117306', 'weight': 1}
    folium.GeoJson(merge, name = 'Cities', style_function= lambda x: style_cities).add_to(map)

    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states, name = 'States', style_function= lambda x: style_states).add_to(map)
    
    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)
    
    folium.Choropleth(geo_data=merge.to_json(),
                  name='Choropleth',
                  data=merge,
                  columns=['GEOCODIBGE', 'AREAMUNKM'],
                  key_on = 'feature.properties.GEOCODIBGE',
                  fill_color = 'Reds',
                  nan_fill_color = 'white',
                  highlight = True,
                  legend_name='Affected Area in km²').add_to(map)

    marker_cluster = MarkerCluster().add_to(map)
    marker_cluster = folium_add_markers(marker_cluster, merge, merge, 2, alerts_df, 'GEOCODIBGE', 'NM_MUN', texts['total_dmg'], 'AREAMUNKM')
    folium.LayerControl().add_to(map)

    return map

In [263]:
lst_states = list(df_states['SIGLA_UF'].unique())
lst_states

['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']

In [264]:
dict_classes, texts = get_texts()

for i in range(len(lst_states)):
    filter = []
    filter.append(lst_states[i])
    map = cities_map(filter)
    map_name = 'Cities_EN_' + filter[0]
    save_map(map_name,map)
    print(map_name + ' saved.')

/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_AC saved.


/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_AM saved.


/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


...

Cities_EN_RO saved.


/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_RR saved.


/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


Cities_EN_TO saved.


In [265]:
dict_classes, texts = get_texts()
map = cities_map([])
map_name = 'All_Cities_EN'
save_map(map_name,map)
print(map_name + ' saved.')

/var/folders/mk/np6rblqn5jzdcb48rl2zt3bm0000gn/T/ipykernel_37544/263644965.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge['AREAMUNKM'].fillna(0, inplace=True)


All_Cities_EN saved.


### C Units

In [280]:
def normalize_string(s):
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^\w\s]', '', s)
    s = s.upper()
    return s

def c_units_map():
    ############# Data Preparation #############
    c_units_copy = c_units.copy()
    c_units_copy['UC'] = c_units_copy['UC'].apply(normalize_string)
    
    alerts_uc = alerts_df[alerts_df['UC'].notna()].copy()
    dic_correcao = {'FLORESTA NACIONAL DE ALTAMIRA': 'FLORESTA NACIONAL ALTAMIRA', 
                    'FLORESTA NACIONAL DE CAXIUANÂ': 'FLORESTA NACIONAL DE CAXIUANÃ', 
                    'FLORESTA NACIONAL DO AMANA': 'FLORESTA NACIONAL DO AMANÁ',
                    'FLORESTA NACIONAL DO BOM FUTURO': 'FLORESTA NACIONAL DE BOM FUTURO',
                    'FLORESTA NACIONAL DO ITACAIUNAS': 'FLORESTA NACIONAL DE ITACAIUNAS',
                    'FLORESTA NACIONAL DO JATUARANA': 'FLORESTA NACIONAL DE JATUARANA',
                    'FLORESTA NACIONAL DO PURUS': 'RESERVA EXTRATIVISTA DO MÉDIO PURÚS',
                    'FLORESTA NACIONAL DO TAPAJÓS': 'FLORESTA NACIONAL DE TAPAJÓS',
                    'FLORESTA NACIONAL DO TAPIRAPÉAQUIRI': 'FLORESTA NACIONAL DE TAPIRAPÉAQUIRI',
                    'FLORESTA NACIONAL MAPIÁ  INAUINI': 'FLORESTA NACIONAL DE MAPIÁINAUINÍ',
                    'PARQUE NACIONAL SERRA DA CUTIA': 'PARQUE NACIONAL DA SERRA DA CUTIA',
                    'RESERVA BIOLÓGICA NASCENTES DA SERRA DO CACHIMBO': 'RESERVA BIOLÓGICA NASCENTES SERRA DO CACHIMBO',
                    'RESERVA EXTRATIVISTA DO ALTO JURUÁ': 'RESERVA EXTRATIVISTA ALTO JURUÁ',
                    'RESERVA EXTRATIVISTA DO ALTO TARAUACÁ': 'RESERVA EXTRATIVISTA ALTO TARAUACÁ',
                    'RESERVA EXTRATIVISTA DO BAIXO JURUÁ': 'RESERVA EXTRATIVISTA BAIXO JURUÁ',
                    'RESERVA EXTRATIVISTA DO CIRIACO': 'RESERVA EXTRATIVISTA DO CIRIÁCO',
                    'RESERVA EXTRATIVISTA DO LAGO DO CUNIÃ': 'RESERVA EXTRATIVISTA LAGO DO CUNIÃ',
                    'RESERVA EXTRATIVISTA DO MÉDIO JURUÁ': 'RESERVA EXTRATIVISTA MÉDIO JURUÁ',
                    'RESERVA EXTRATIVISTA DO RIO CAJARI': 'RESERVA EXTRATIVISTA RIO CAJARI',
                    'RESERVA EXTRATIVISTA DO RIO DO CAUTÁRIO': 'RESERVA EXTRATIVISTA RIO CAUTÁRIO',
                    'RESERVA EXTRATIVISTA DO RIO OURO PRETO': 'RESERVA EXTRATIVISTA RIO OURO PRETO',
                    'RESERVA EXTRATIVISTA RIO UNINI': 'RESERVA EXTRATIVISTA DO RIO UNINI',
                    'RESERVA EXTRATIVISTA TAPAJÓSARAPIUNS': 'RESERVA EXTRATIVISTA TAPAJÓS ARAPIUNS',
                    'RESERVA EXTRATIVISTA TAPAJÓS-ARAPIUNS': 'RESERVA EXTRATIVISTA TAPAJÓS ARAPIUNS',
                    'RESERVA EXTRATIVISTA TERRA GRANDE  PRACUÚBA': 'RESERVA EXTRATIVISTA TERRA GRANDE PRACUUBA',
                    'RESERVA EXTRATIVISTA TERRA GRANDE - PRACUÚBA': 'RESERVA EXTRATIVISTA TERRA GRANDE PRACUUBA',
                    'ÁREA DE PROTEÇÃO AMBIENTAL DOS MEANDROS DO RIO ARAGUAIA': 'ÁREA DE PROTEÇÃO AMBIENTAL MEANDROS DO ARAGUAIA',
                    'ÁREA DE RELEVANTE INTERESSE ECOLÓGICO SERINGAL NOVA ESPERANÇA': 'ÁREA DE RELEVANTE INTERESSE ECOLÓGICA SERINGAL NOVA ESPERANÇA',
                    'ESTAÇÃO ECOLÓGICA JUAMI-JAPURÁ': 'ESTAÇÃO ECOLÓGICA JUAMIJAPURÁ',
                    'FLORESTA NACIONAL DE BALATA-TUFARI': 'FLORESTA NACIONAL DE BALATATUFARI',
                    'FLORESTA NACIONAL DE SARACÁ-TAQUERA': 'FLORESTA NACIONAL DE SARACÁTAQUERA',
                    'FLORESTA NACIONAL MAPIÁ - INAUINI': 'FLORESTA NACIONAL DE MAPIÁINAUINÍ',
                    'RESERVA EXTRATIVISTA AUATÍ-PARANÁ': 'RESERVA EXTRATIVISTA AUATÍPARANÁ',
                    'RESERVA EXTRATIVISTA DO CAZUMBÁ-IRACEMA': 'RESERVA EXTRATIVISTA DO CAZUMBÁIRACEMA',
                    'RESERVA EXTRATIVISTA GURUPÁ-MELGAÇO': 'RESERVA EXTRATIVISTA GURUPÁMELGAÇO',
                    'RESERVA EXTRATIVISTA IPAÚ-ANILZINHO': 'RESERVA EXTRATIVISTA IPAÚANILZINHO'}
    alerts_uc['UC'] = alerts_uc['UC'].replace(dic_correcao)
    gc_uc = alerts_uc.groupby('UC')['AREAMUNKM'].sum().reset_index()
    gc_uc['UC'] = gc_uc['UC'].replace(dic_correcao)
    

    def uc_geodf(state):
        first_alert = alerts_df[alerts_df['UC'] == state]['geometry'].iloc[0]
        representative_point = first_alert
        new_record = {
            'UC': state,
            'geometry': representative_point
        }
        return gpd.GeoDataFrame([new_record], crs=c_units_copy.crs)
    
    c_units_copy = pd.concat([c_units_copy, uc_geodf('ESTAÇÃO ECOLÓGICA DE CARACARAÍ')], ignore_index=True)
    c_units_copy = pd.concat([c_units_copy, uc_geodf('ESTAÇÃO ECOLÓGICA DE IQUÊ')], ignore_index=True)
    
    merge_ucs = pd.merge(c_units_copy, gc_uc, on='UC', how='left').fillna(0)


    
    #############       Folium       #############

    map = folium_map_init()

    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)

    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states, name = 'States', style_function= lambda x: style_states).add_to(map)

    style_ucs = {'fillOpacity':0 ,'color' : '#3d1601', 'weight': 1}
    folium.GeoJson(c_units_copy, name = 'Conservation Units', style_function= lambda x: style_ucs).add_to(map)


    folium.Choropleth(geo_data=merge_ucs.to_json(),
                  name='Choropleth',
                  data=merge_ucs,
                  columns=['UC', 'AREAMUNKM'],
                  key_on = 'feature.properties.UC',
                  fill_color = 'YlOrRd',
                  nan_fill_color = 'white',
                  highlight = True,
                  legend_name='Affected Area in km²').add_to(map)
    
    marker_cluster = MarkerCluster().add_to(map)

    marker_cluster = folium_add_markers(marker_cluster,merge_ucs,merge_ucs, 2, alerts_uc, 'UC', 'UC', texts['total_dmg'], 'AREAMUNKM')

    folium.LayerControl().add_to(map)
    return map

In [281]:
dict_classes, texts = get_texts()
map = c_units_map()
save_map('C_Units_EN',map)